In [70]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
from io import BytesIO
import scipy.linalg

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_bloch_multivector

def simulate_hamiltonian_precession_stepped():
    print("Starting step-by-step Hamiltonian simulation... This may take a moment.")

    steps = 50  
    omega_z = 5.0
    omega_x = 0.0

    total_time = 2 * np.pi
    dt = total_time / (steps - 1)

    output_filename = "larmor_precession_hamiltonian_stepped.gif"
    images = []
    z = Operator.from_label('Z')
    x = Operator.from_label('X')
    hamiltonian = omega_z * z + omega_x * x

    qc_initial = QuantumCircuit(1)
    initial_angle = np.pi / 4
    qc_initial.ry(initial_angle, 0)
    initial_state = Statevector.from_instruction(qc_initial)

    print("Simulating time evolution step-by-step...")

    unitary_step_matrix = scipy.linalg.expm(-1j * hamiltonian.data * dt)
    unitary_step_op = Operator(unitary_step_matrix)

    current_state = initial_state.copy()

    for i in range(steps):
        t = i * dt
        fig = plot_bloch_multivector(current_state, title=f"Hamiltonian Evolution (Stepped)\nTime = {t:.2f}")

        buf = BytesIO()
        fig.savefig(buf, format='png', dpi=100)
        buf.seek(0)
        images.append(imageio.imread(buf))
        plt.close(fig)
        
        print(f"Generated frame {i+1}/{steps} for t = {t:.2f}")
        current_state = current_state.evolve(unitary_step_op)

    print(f"\nStitching frames together to create {output_filename}...")
    imageio.mimsave(output_filename, images, fps=15)

    print("\nSimulation complete!")
    print(f"Animation saved as '{output_filename}' in the current directory.")


if __name__ == '__main__':
    try:
        from qiskit_aer import Aer
    except ImportError:
        print("qiskit-aer not found, using default Aer provider.")
        from qiskit.providers.aer import Aer

    simulate_hamiltonian_precession_stepped()



Starting step-by-step Hamiltonian simulation... This may take a moment.
Simulating time evolution step-by-step...


C:\Users\Dell\AppData\Local\Temp\ipykernel_33672\4081062158.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(buf))


Generated frame 1/50 for t = 0.00
Generated frame 2/50 for t = 0.13
Generated frame 3/50 for t = 0.26
Generated frame 4/50 for t = 0.38
Generated frame 5/50 for t = 0.51
Generated frame 6/50 for t = 0.64
Generated frame 7/50 for t = 0.77
Generated frame 8/50 for t = 0.90
Generated frame 9/50 for t = 1.03
Generated frame 10/50 for t = 1.15
Generated frame 11/50 for t = 1.28
Generated frame 12/50 for t = 1.41
Generated frame 13/50 for t = 1.54
Generated frame 14/50 for t = 1.67
Generated frame 15/50 for t = 1.80
Generated frame 16/50 for t = 1.92
Generated frame 17/50 for t = 2.05
Generated frame 18/50 for t = 2.18
Generated frame 19/50 for t = 2.31
Generated frame 20/50 for t = 2.44
Generated frame 21/50 for t = 2.56
Generated frame 22/50 for t = 2.69
Generated frame 23/50 for t = 2.82
Generated frame 24/50 for t = 2.95
Generated frame 25/50 for t = 3.08
Generated frame 26/50 for t = 3.21
Generated frame 27/50 for t = 3.33
Generated frame 28/50 for t = 3.46
Generated frame 29/50 for t =

In [ ]:
def simulate_gradient_precession():
    print("Starting simulation of precession in a magnetic field gradient...")

    steps = 60
    total_time = 4.0

    B0 = 1.0
    alpha = 2.0

    num_points = 101
    z_max = 2.0
    z_grid = np.linspace(-z_max, z_max, num_points)

    dt = total_time / (steps - 1)
    output_filename_dephasing = "gradient_precession.gif"
    output_filename_bloch = "gradient_bloch_sphere.gif"
    images_dephasing = []
    images_bloch = []

    pauli_x = np.array([[0, 1], [1, 0]])
    pauli_y = np.array([[0, -1j], [1j, 0]])
    pauli_z = np.array([[1, 0], [0, -1]])

    H = np.zeros((2 * num_points, 2 * num_points), dtype=complex)
    for i, z_val in enumerate(z_grid):
        local_H = (B0 + alpha * z_val) * pauli_z
        H[2*i:2*(i+1), 2*i:2*(i+1)] = local_H
    spatial_wavefunc = np.exp(-z_grid**2 / 2.0)
    spatial_wavefunc /= np.linalg.norm(spatial_wavefunc)
    spin_state = np.array([1/np.sqrt(2), -1/np.sqrt(2)], dtype=complex)
    initial_state = np.kron(spatial_wavefunc, spin_state)

    X_full = np.kron(np.identity(num_points), pauli_x)
    Y_full = np.kron(np.identity(num_points), pauli_y)
    Z_full = np.kron(np.identity(num_points), pauli_z)

    unitary_step = scipy.linalg.expm(-1j * H * dt)
    current_state = initial_state.copy()

    for i in range(steps):
        t = i * dt
        print(f"Generated frame {i+1}/{steps} for t = {t:.2f}")
        state_grid = current_state.reshape((num_points, 2))
        exp_X_spatial = np.einsum('ij,jk,ik->i', state_grid.conj(), pauli_x, state_grid).real
        exp_Y_spatial = np.einsum('ij,jk,ik->i', state_grid.conj(), pauli_y, state_grid).real
        
        fig1, ax1 = plt.subplots(figsize=(8, 6))
        ax1.plot(z_grid, exp_X_spatial, label='<X>')
        ax1.plot(z_grid, exp_Y_spatial, label='<Y>')
        ax1.set_ylim(-1.1, 1.1)
        ax1.set_xlim(-z_max, z_max)
        ax1.set_xlabel("Position (z)")
        ax1.set_ylabel("Spin Expectation Value")
        ax1.set_title(f"Spin Dephasing in a Field Gradient\nTime = {t:.2f}")
        ax1.legend()
        ax1.grid(True)
        
        buf1 = BytesIO()
        fig1.savefig(buf1, format='png', dpi=100)
        buf1.seek(0)
        images_dephasing.append(imageio.imread(buf1))
        plt.close(fig1)

        avg_exp_X = (current_state.conj().T @ X_full @ current_state).real
        avg_exp_Y = (current_state.conj().T @ Y_full @ current_state).real
        avg_exp_Z = (current_state.conj().T @ Z_full @ current_state).real
        bloch_vector = [avg_exp_X, avg_exp_Y, avg_exp_Z]

        fig2 = plot_bloch_vector(bloch_vector, title=f"Average Bloch Vector\nTime = {t:.2f}")
        
        buf2 = BytesIO()
        fig2.savefig(buf2, format='png', dpi=100)
        buf2.seek(0)
        images_bloch.append(imageio.imread(buf2))
        plt.close(fig2)

        current_state = unitary_step @ current_state

    print(f"\nCreating GIFs...")
    imageio.mimsave(output_filename_dephasing, images_dephasing, fps=15)
    print(f"Saved spatial dephasing animation as '{output_filename_dephasing}'")
    imageio.mimsave(output_filename_bloch, images_bloch, fps=15)
    print(f"Saved Bloch sphere animation as '{output_filename_bloch}'")
    print("\nSimulation complete!")

if __name__ == '__main__':
    simulate_gradient_precession()



Starting simulation of precession in a magnetic field gradient...
Generated frame 1/60 for t = 0.00


C:\Users\Dell\AppData\Local\Temp\ipykernel_33672\1961324.py:96: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images_dephasing.append(imageio.imread(buf1))
C:\Users\Dell\AppData\Local\Temp\ipykernel_33672\1961324.py:111: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images_bloch.append(imageio.imread(buf2))


Generated frame 2/60 for t = 0.07
Generated frame 3/60 for t = 0.14
Generated frame 4/60 for t = 0.20
Generated frame 5/60 for t = 0.27
Generated frame 6/60 for t = 0.34
Generated frame 7/60 for t = 0.41
Generated frame 8/60 for t = 0.47
Generated frame 9/60 for t = 0.54
Generated frame 10/60 for t = 0.61
Generated frame 11/60 for t = 0.68
Generated frame 12/60 for t = 0.75
Generated frame 13/60 for t = 0.81
Generated frame 14/60 for t = 0.88
Generated frame 15/60 for t = 0.95
Generated frame 16/60 for t = 1.02
Generated frame 17/60 for t = 1.08
Generated frame 18/60 for t = 1.15
Generated frame 19/60 for t = 1.22
Generated frame 20/60 for t = 1.29
Generated frame 21/60 for t = 1.36
Generated frame 22/60 for t = 1.42
Generated frame 23/60 for t = 1.49
Generated frame 24/60 for t = 1.56
Generated frame 25/60 for t = 1.63
Generated frame 26/60 for t = 1.69
Generated frame 27/60 for t = 1.76
Generated frame 28/60 for t = 1.83
Generated frame 29/60 for t = 1.90
Generated frame 30/60 for t 